In [1]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.utils import to_dense_adj

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # print("x after linear", x, x.ndimension())
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [11]:
dataset = CVFConfigDataset()
data_loader = DataLoader(dataset, batch_size=1024, shuffle=False)

In [12]:
# edge_index = torch.tensor([
#     [0, 1],
#     [0, 2],
#     [0, 3],
#     [1, 0],
#     [2, 0],
#     [3, 0]
# ], dtype=torch.long)
edge_index = torch.tensor(
    [
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [0, 7],
        [0, 8],
        [0, 9],
        [1, 0],
        [2, 0],
        [3, 0],
        [4, 0],
        [5, 0],
        [6, 0],
        [7, 0],
        [8, 0],
        [9, 0],
    ],
    dtype=torch.long,
)


# # x = torch.tensor([[0], [0], [0], [0]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]
# x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

# y = torch.tensor([[3.0, 1.0]]) # ranks

# data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)
# data.train_mask = torch.tensor([1 for _ in range(len(x))])

In [13]:
adjacency = to_dense_adj(edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
# adjacency = adjacency.repeat(data_loader.batch_size, 1)
adjacency = adjacency.unsqueeze(0)
adjacency

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]])

In [14]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_out)
        # self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        h = torch.relu(h)
        # print("h2 r", h)
        # h = self.out(h)
        # print(torch.relu(h))
        # result = F.softmax(h, dim=0)
        # print("result", result)
        # print("h", h)
        h = h.squeeze(-1)
        # print("h", h)
        # h = h.flatten().unsqueeze(1)
        # print("h", h)
        return h

    def fit(self, data_loader, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs + 1):
            avg_loss = 0
            count = 0
            for batch in data_loader:
                # x = batch[0].squeeze(1)
                # print(batch[0])
                # print(x)
                x = batch[0]
                # print(x, x.shape)
                # y = torch.zeros(4, 1)
                # y[batch[1]] = 1.0
                y = batch[1]
                optimizer.zero_grad()
                out = self(x, adjacency)
                # print(out)
                # print(out, y)
                # print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Loss:", avg_loss / count)

            # if loss < 0.009:
            #     print("Loss threshold met.")
            #     break

In [15]:
gnn = VanillaGNN(1, 64, 1)
print(gnn)
gnn.fit(data_loader, epochs=50)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=1, out_features=64, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=64, out_features=1, bias=False)
  )
)
Loss: tensor(1.3773, grad_fn=<DivBackward0>)
Loss: tensor(1.2331, grad_fn=<DivBackward0>)
Loss: tensor(1.5914, grad_fn=<DivBackward0>)
Loss: tensor(1.1852, grad_fn=<DivBackward0>)
Loss: tensor(0.8731, grad_fn=<DivBackward0>)
Loss: tensor(0.6668, grad_fn=<DivBackward0>)
Loss: tensor(0.7727, grad_fn=<DivBackward0>)
Loss: tensor(0.7211, grad_fn=<DivBackward0>)
Loss: tensor(0.6509, grad_fn=<DivBackward0>)
Loss: tensor(0.6885, grad_fn=<DivBackward0>)
Loss: tensor(0.6831, grad_fn=<DivBackward0>)
Loss: tensor(0.6641, grad_fn=<DivBackward0>)
Loss: tensor(0.6800, grad_fn=<DivBackward0>)
Loss: tensor(0.6738, grad_fn=<DivBackward0>)
Loss: tensor(0.6730, grad_fn=<DivBackward0>)
Loss: tensor(0.6786, grad_fn=<DivBackward0>)
Loss: tensor(0.6725, grad_fn=<DivBackward0>)
Loss: tensor(0.6757, grad_fn=

In [9]:
# out = gnn(torch.tensor([[3.], [2.], [2.], [2.]]), adjacency)
# print(out)

In [10]:
for batch in data_loader:
    x = batch[0]
    y = batch[1]
    predicted = gnn(x, adjacency).argmax(dim=1)
    print("y", y, "predicted", predicted)

y tensor([9, 8, 8, 7, 8, 7, 7, 6, 8, 7, 7, 6, 7, 6, 6, 5, 8, 7, 7, 6, 7, 6, 6, 5,
        7, 6, 6, 5, 6, 5, 5, 4, 8, 7, 7, 6, 7, 6, 6, 5, 7, 6, 6, 5, 6, 5, 5, 4,
        7, 6, 6, 5, 6, 5, 5, 4, 6, 5, 5, 4, 5, 4, 4, 3, 8, 7, 7, 6, 7, 6, 6, 5,
        7, 6, 6, 5, 6, 5, 5, 4, 7, 6, 6, 5, 6, 5, 5, 4, 6, 5, 5, 4, 5, 4, 4, 3,
        7, 6, 6, 5, 6, 5, 5, 4, 6, 5, 5, 4, 5, 4, 4, 3, 6, 5, 5, 4, 5, 4, 4, 3,
        5, 4, 4, 3, 4, 3, 3, 2, 8, 7, 7, 6, 7, 6, 6, 5, 7, 6, 6, 5, 6, 5, 5, 4,
        7, 6, 6, 5, 6, 5, 5, 4, 6, 5, 5, 4, 5, 4, 4, 3, 7, 6, 6, 5, 6, 5, 5, 4,
        6, 5, 5, 4, 5, 4, 4, 3, 6, 5, 5, 4, 5, 4, 4, 3, 5, 4, 4, 3, 4, 3, 3, 2,
        7, 6, 6, 5, 6, 5, 5, 4, 6, 5, 5, 4, 5, 4, 4, 3, 6, 5, 5, 4, 5, 4, 4, 3,
        5, 4, 4, 3, 4, 3, 3, 2, 6, 5, 5, 4, 5, 4, 4, 3, 5, 4, 4, 3, 4, 3, 3, 2,
        5, 4, 4, 3, 4, 3, 3, 2, 4, 3, 3, 2, 3, 2, 2, 1, 8, 7, 7, 6, 7, 6, 6, 5,
        7, 6, 6, 5, 6, 5, 5, 4, 7, 6, 6, 5, 6, 5, 5, 4, 6, 5, 5, 4, 5, 4, 4, 3,
        7, 6, 6, 5, 6, 5, 5, 4, 6, 5, 